In [1]:
import copy
import os
import time
import random
import sys
import warnings

import numpy as np
import pandas as pd
from typing import Any


from hyperimpute.plugins.imputers import Imputers
from hyperimpute.utils.distributions import enable_reproducible_results
import hyperimpute.logger as log

from benchmark_imputation import evaluate_dataset_repeated_internal

from IPython.display import HTML, display
import tabulate

warnings.filterwarnings('ignore')
enable_reproducible_results()

imputers = Imputers()
log.add(sink=sys.stderr, level="INFO")

In [2]:
def get_imputer():
    return imputers.get("hyperimpute", 
        optimizer = "simple"
    )

def evaluate_dataset_repeated(
    name,
    X_raw,
    y,
    ref_methods=["mean", "sklearn_missforest", "sklearn_ice", "gain", "sinkhorn", "softimpute"],
    scenarios=["MNAR", "MCAR", "MAR"],
    miss_pct=[0.1, 0.3, 0.5, 0.7],
    n_iter=2,
    debug=False,
):
    return evaluate_dataset_repeated_internal(
        name = name,
        evaluated_model = get_imputer(),
        X_raw = X_raw,
        y = y,
        ref_methods=ref_methods,
        scenarios=scenarios,
        miss_pct=miss_pct,
        n_iter=n_iter,
        debug=debug,
    )


## Sanity check in  debug mode

In [3]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00291/airfoil_self_noise.dat', header = None, sep ="\\t")

y = df[5]
X_raw = df.drop(columns = [5])

evaluate_dataset_repeated("airfoil", X_raw, y,        
            scenarios =  ["MNAR"],      
            ref_methods=["mean"],
            debug = True,
            n_iter = 1,
            miss_pct = [0.3, 0.5]
)

> evaluation trial  0
  > eval  MNAR 0.3


[2022-01-10T11:55:08.501278+0200][31115][INFO]   > BO iter 0
[2022-01-10T11:55:09.045764+0200][31115][INFO]      >>> Column 0 <-- score -0.02205200381533318 <-- Model random_forest_regressor
[2022-01-10T11:55:09.474049+0200][31115][INFO]      >>> Column 1 <-- score -0.024103476543516647 <-- Model catboost_regressor
[2022-01-10T11:55:11.165393+0200][31115][INFO]      >>> Column 2 <-- score 0.9467684 <-- Model catboost
[2022-01-10T11:55:12.367450+0200][31115][INFO]      >>> Column 3 <-- score 0.81683 <-- Model catboost
[2022-01-10T11:55:12.758875+0200][31115][INFO]      >>> Column 4 <-- score -0.01805905811542978 <-- Model catboost_regressor
[2022-01-10T11:55:55.692778+0200][31115][INFO]   > BO iter 1
[2022-01-10T11:55:57.080857+0200][31115][INFO]      >>> Column 3 <-- score 0.9173439999999999 <-- Model catboost
[2022-01-10T11:55:57.506701+0200][31115][INFO]      >>> Column 4 <-- score -0.00014304528998656266 <-- Model xgboost_regressor
[2022-01-10T11:55:59.039961+0200][31115][INFO]     

  > eval  MNAR 0.5


[2022-01-10T11:58:33.839548+0200][31115][INFO]      >>> Column 0 <-- score -0.028120643458368982 <-- Model random_forest_regressor
[2022-01-10T11:58:34.267790+0200][31115][INFO]      >>> Column 1 <-- score -0.029424511091093154 <-- Model catboost_regressor
[2022-01-10T11:58:35.060886+0200][31115][INFO]      >>> Column 2 <-- score 0.8449091601065641 <-- Model catboost
[2022-01-10T11:58:35.719576+0200][31115][INFO]      >>> Column 3 <-- score 0.633037342816909 <-- Model xgboost
[2022-01-10T11:58:36.091867+0200][31115][INFO]      >>> Column 4 <-- score -0.029780313809634813 <-- Model catboost_regressor
[2022-01-10T11:59:11.066358+0200][31115][INFO]   > BO iter 1
[2022-01-10T11:59:12.011021+0200][31115][INFO]      >>> Column 2 <-- score 0.9954343021586576 <-- Model catboost
[2022-01-10T11:59:12.479834+0200][31115][INFO]      >>> Column 1 <-- score -0.0007007482960861136 <-- Model xgboost_regressor
[2022-01-10T11:59:13.255994+0200][31115][INFO]      >>> Column 3 <-- score 0.8357438246627417

RMSE score


Scenario,"miss_pct [0, 1]",Our method,mean
MNAR,0.3,0.215636,0.291261
MNAR,0.5,0.28744,0.293272




Wasserstein score


Scenario,"miss_pct [0, 1]",Our method,mean
MNAR,0.3,0.049091,0.368713
MNAR,0.5,0.0767515,0.581678




Downstream model prediction error


Scenario,"miss_pct [0, 1]",Our method,mean
MNAR,0.3,5.68334,6.11345
MNAR,0.5,15.4386,17.8344


# Datasets

In [4]:
import hyperimpute.logger as log

log.remove()

|           Dataset          |  Size |
|:--------------------------:|:-----:|
|     airfoil self noise     |  1503 |
|     blood transfusion      |  748  |
|  breast cancer diagnostic  |  569  |
|         california         | 20640 |
|   climate model crashes    |  540  |
|    concrete compression    |  1030 |
|       concrete slump       |  103  |
| connectionist bench sonar  |  208  |
|  connectionist bench vowel |  990  |
|            iris            |  150  |
|      wine quality red      |  1599 |
|     wine quality white     |  4899 |
|            yeast           |  1484 |

## Dataset: UCI Airfoil Self-Noise Data Set

https://archive.ics.uci.edu/ml/datasets/airfoil+self-noise


In [5]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00291/airfoil_self_noise.dat', header = None, sep ="\\t")

df

,0,1,2,3,4,5
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461
...,...,...,...,...,...,...
1498,2500,15.6,0.1016,39.6,0.052849,110.264
1499,3150,15.6,0.1016,39.6,0.052849,109.254
1500,4000,15.6,0.1016,39.6,0.052849,106.604
1501,5000,15.6,0.1016,39.6,0.052849,106.224


In [ ]:
last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns = [last_col])

evaluate_dataset_repeated("airfoil", X_raw, y)

scenario failed 


## Dataset: UCI Blood Transfusion Service Center Data Set

https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data

In [ ]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data')

df

In [ ]:
last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns = [last_col])

evaluate_dataset_repeated("blood", X_raw, y)

## Dataset: Breast Cancer Wisconsin (Diagnostic)

https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)

In [ ]:
from sklearn.datasets import load_breast_cancer

X_raw, y = load_breast_cancer(as_frame = True, return_X_y = True)

X_raw

In [ ]:
evaluate_dataset_repeated("bc", X_raw, y)

## Dataset: California Housing



In [ ]:
from sklearn.datasets import fetch_california_housing

X_raw, y = fetch_california_housing(as_frame = True, return_X_y = True)

X_raw

In [ ]:
evaluate_dataset_repeated("california", X_raw, y)

# Dataset: Climate Model Simulation Crashes
https://archive.ics.uci.edu/ml/datasets/climate+model+simulation+crashes

In [ ]:
samples = np.loadtxt("https://archive.ics.uci.edu/ml/machine-learning-databases/00252/pop_failures.dat", skiprows=1)
df = pd.DataFrame(samples)

df

In [ ]:
last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns = [last_col])

evaluate_dataset_repeated("climate_model", X_raw, y)

## Concrete Compressive Strength Data Set
https://archive.ics.uci.edu/ml/datasets/concrete+compressive+strength

In [ ]:
df = pd.read_excel("https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls")

df

In [ ]:
last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns = [last_col])

evaluate_dataset_repeated("concrete_compressive", X_raw, y)

## Concrete Slump Test Data Set

https://archive.ics.uci.edu/ml/datasets/concrete+slump+test

In [ ]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/slump/slump_test.data")

df

In [ ]:
last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns = ["No", last_col])

evaluate_dataset_repeated("concret_slump", X_raw, y)

## Connectionist Bench (Sonar, Mines vs. Rocks) Data Set

https://archive.ics.uci.edu/ml/datasets/connectionist+bench+(sonar,+mines+vs.+rocks)

In [ ]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data", header = None)

df

In [ ]:
last_col = df.columns[-1]
y = (df[last_col] == 'M').astype(int)
X_raw = df.drop(columns = [last_col])

evaluate_dataset_repeated("connectionist_sonar", X_raw, y)

## Wine-Red dataset

In [ ]:
# Wine Quality Data Set

df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep = ';')

df

In [ ]:
last_col = df.columns[-1]

y = df[last_col]
mapped_labels = sorted(y.unique())
mapping = {}
for idx, label in enumerate(mapped_labels):
    mapping[label] = idx
y = y.map(mapping)       

X_raw = df.drop(columns = [last_col])

evaluate_dataset_repeated("wine_red", X_raw, y)

## Wine-White dataset

In [ ]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv", sep = ';')

df

In [ ]:
last_col = df.columns[-1]

y = df[last_col]
mapped_labels = sorted(y.unique())
mapping = {}
for idx, label in enumerate(mapped_labels):
    mapping[label] = idx
y = y.map(mapping)  

X_raw = df.drop(columns = [last_col])

evaluate_dataset_repeated("wine_white", X_raw, y)

## Yeast Data Set


In [ ]:
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/yeast/yeast.data",sep="\s+", header = None)

df = df.drop(columns = [0])

for col in [9]:
    df[col] = LabelEncoder().fit_transform(df[col])

df

In [ ]:
last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns = [last_col])

evaluate_dataset_repeated("yeast", X_raw, y)

## Diabetes


In [ ]:
from sklearn.datasets import load_diabetes

X, y = load_diabetes(as_frame = True, return_X_y = True)

X

In [ ]:
evaluate_dataset_repeated("diabetes", X, y)

## Iris


In [ ]:
from sklearn.datasets import load_iris

X, y = load_iris(as_frame = True, return_X_y = True)

X

In [ ]:
evaluate_dataset_repeated("iris", X, y)

# Conclusion
